In [1]:
import fastaparser as fst
import pandas as pd
import re

In [2]:
contig_path = "trim/Poil_contig.fa"
scaffold_path = "trim/Poil_scaffold.fa"
gapClosed_path = "trim/Poil_gapClosed.fa"

In [3]:
# Парсим fasta в Pandas Dataframe
def parser(path):
    with open(path, "r") as file:
        seq_parser = fst.Reader(file, parse_method='quick')
        df = pd.DataFrame(columns=["Sequence", "Length"])
        for seq in seq_parser:
            df.loc[seq.header, 'Sequence'] = seq.sequence
            df.loc[seq.header, 'Length'] = len(seq.sequence)
    return df.sort_values(by="Length", ascending=False)

def general_stats(df):
    print("Number of sequences:", len(df))
    print("Total length:", df.Length.sum())
    print("The size of the longest sequence:", df.Length[0])
    i = 1; len_sum = df.Length[0]
    while len_sum < df.Length.sum() / 2:
        len_sum += df.Length[i]
        i += 1
    print("N50:", df.Length[i-1])
    
def gaps(seq):
    print("Number of gaps:", len(re.findall("N+", seq)))
    print("The gaps themselves: ", re.findall("N+", seq))
    print("Total length of gaps:", seq.count("N"))

In [4]:
contigs = parser(contig_path)
scaffold = parser(scaffold_path)
gapClosed = parser(gapClosed_path)

In [5]:
print("\tStatistics for contigs"); general_stats(contigs)
print("\n\tStatistics for scaffolds"); general_stats(scaffold); gaps(scaffold.Sequence[0])
print("\n\tStatistics for gapClosed"); general_stats(gapClosed); gaps(gapClosed.Sequence[0])

	Statistics for contigs
Number of sequences: 609
Total length: 3925016
The size of the longest sequence: 179307
N50: 47798

	Statistics for scaffolds
Number of sequences: 95
Total length: 3870305
The size of the longest sequence: 383603
N50: 173397
Number of gaps: 3
The gaps themselves:  ['N', 'NNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNNNNNNNNN']
Total length of gaps: 40

	Statistics for gapClosed
Number of sequences: 95
Total length: 3870322
The size of the longest sequence: 383574
N50: 173397
Number of gaps: 0
The gaps themselves:  []
Total length of gaps: 0


In [6]:
# Сохраняем самый длинный скаффолд в fasta-файл
with open(r"trim/longest.fasta", 'w') as fasta_file:
    writer = fst.Writer(fasta_file)
    writer.writefasta((gapClosed.index[0], gapClosed.Sequence[0]))

# Дополнительное задание
### Взяли на 20% меньше ридов - 4 млн и 1.2 млн вместо 5 и 1.5 соответственно

In [7]:
s_contigs = parser("small/"+contig_path)
s_scaffold = parser("small/"+scaffold_path)
s_gapClosed = parser("small/"+gapClosed_path)

In [8]:
print("\tStatistics for contigs"); general_stats(s_contigs)
print("\n\tStatistics for scaffolds"); general_stats(s_scaffold); gaps(s_scaffold.Sequence[0])
print("\n\tStatistics for gapClosed"); general_stats(s_gapClosed); gaps(s_gapClosed.Sequence[0])

	Statistics for contigs
Number of sequences: 630
Total length: 3924487
The size of the longest sequence: 189755
N50: 58446

	Statistics for scaffolds
Number of sequences: 100
Total length: 3868693
The size of the longest sequence: 383606
N50: 173393
Number of gaps: 4
The gaps themselves:  ['NNNNN', 'NNNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNNNN']
Total length of gaps: 69

	Statistics for gapClosed
Number of sequences: 100
Total length: 3868817
The size of the longest sequence: 383570
N50: 173393
Number of gaps: 0
The gaps themselves:  []
Total length of gaps: 0


Как видим, результат чуть хуже, но не критично. Значения N50 отличаются незначительно, как и остальные статистики

### Взяли на 50% меньше ридов - 2.5 млн и 0.75 млн вместо 5 и 1.5 соответственно

In [9]:
ss_contigs = parser("smallest/"+contig_path)
ss_scaffold = parser("smallest/"+scaffold_path)
ss_gapClosed = parser("smallest/"+gapClosed_path)

In [10]:
print("\tStatistics for contigs"); general_stats(ss_contigs)
print("\n\tStatistics for scaffolds"); general_stats(ss_scaffold); gaps(ss_scaffold.Sequence[0])
print("\n\tStatistics for gapClosed"); general_stats(ss_gapClosed); gaps(ss_gapClosed.Sequence[0])

	Statistics for contigs
Number of sequences: 693
Total length: 3923130
The size of the longest sequence: 210856
N50: 81322

	Statistics for scaffolds
Number of sequences: 118
Total length: 3863084
The size of the longest sequence: 383400
N50: 173292
Number of gaps: 5
The gaps themselves:  ['NN', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNN']
Total length of gaps: 109

	Statistics for gapClosed
Number of sequences: 118
Total length: 3863069
The size of the longest sequence: 383412
N50: 173292
Number of gaps: 0
The gaps themselves:  []
Total length of gaps: 0


Удивительно, но если оставить лишь половину всех ридов, результат будет не столь плох. Да, увеличивается количество гэпов, уменьшается длина сиквенсов и N50, но не критично.